In [1]:
from imports.ExtractContactCases import ExtractContactCases

ex = ExtractContactCases(
    '/home/hussain/me/projects/tactile/data/contact_extraction9/', 
    bag_file_name='/home/hussain/me/projects/tactile/data/dataset_ENVTACT_new2.bag', augment=False, _keep_raw=True)
ex.extract()

parsing bag: 0it [00:00, ?it/s]

  0%|          | 0/645 [00:00<?, ?it/s]

saving
386 129 129


In [4]:
ex.samples.__len__()

644